In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score
import warnings 
from sklearn import model_selection
from sklearn import linear_model #import LinearRegression, Ridge, RidgeCV, ElasticNet, Lasso, LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('TrainClean.csv',index_col=0)
train.columns

Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_GDP', 'Region_PopulationK', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
       'Mean_Wind_SpeedKm_h', 'Precipitationmm', 'StandardMarket',
       'HyperMarket', 'SuperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'WithNonFoodDepartment', 'Region0', 'Region1',
       'Region2', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'Region9', 'January', 'February', 'March', 'April', 'May',
       'June', 'July', 'August', 'September', 'October', 'November',
       'December', 'Tuesday', 'Wednesday', 'Friday', 'Saturday', 'Monday',
       'Thursday', 'Sunday', 'Snow', 'Fog', 'Thunderstorm', 'Hail', 'Nothing',
       'Rain', 'ZeroClouds', 'H', 'WCI', 'NumberOfSales', 'NumberOfCustomers'],
      dtype='object')

In [4]:
# Simula kfold cross validation per RF
def mycvRF(dataset,fold,trees,rand,variablesC,variablesS,all):
    
    dataset = dataset.sample(frac=1,random_state=rand)
    
    size = dataset.shape[0]
    k = fold
    step = math.floor(size/k) # size for each fold
    
    R2_c = []
    MSE_c = []
    R2_s = []
    MSE_s = []
    
    if all:
        variablesC = dataset.columns.drop('NumberOfCustomers')
        variablesC = variablesC.drop('NumberOfSales')
        variablesS =  dataset.columns.drop('NumberOfSales')
    
      
    for i in range(0,dataset.shape[0],step+1):
        
        left_data = dataset[0:i]
        test_data = dataset[i:i+step]                         #train_data < test_data > train_data
        right_data = dataset[i+step+1:dataset.shape[0]]
        
        
        train_data = left_data.append(right_data)
        train_data = train_data.reset_index(drop=True)

        
    
        var = train_data[variablesC]
        tar = train_data['NumberOfCustomers']
    
        model1 = RandomForestRegressor(random_state=rand,n_jobs =2,min_samples_leaf=5) #model 1 predice customers
        model1.set_params(n_estimators=trees)
        model1.fit(var, tar)
    
        var = train_data[variablesS]
        tar = train_data['NumberOfSales']
    
        model2 = RandomForestRegressor(random_state=rand,n_jobs =2,min_samples_leaf=5) #model 2 predice sales
        model2.set_params(n_estimators=trees)
        model2.fit(var, tar)
    
        ###############################################################################################testing step
    

        NumOfCust = test_data['NumberOfCustomers']  
        NumOfSales = test_data['NumberOfSales']      
    
        first_test = test_data[variablesC]  
    
        Custp = model1.predict(first_test) 
    
        R2_c.append(r2_score(NumOfCust,Custp)) 
        MSE_c.append(mean_squared_error(NumOfCust,Custp)) 
    
        second_test = test_data[variablesS] 
        second_test['NumberOfCustomers'] = Custp 
    
        Salesp = model2.predict(second_test) 
    
        R2_s.append(r2_score(NumOfSales,Salesp))
        MSE_s.append(mean_squared_error(NumOfSales,Salesp));
    
    return R2_c , MSE_c , R2_s , MSE_s

In [5]:
# Simula kfold cross validation per RF
def mycvRFDirect(dataset,fold,trees,rand):
    
    dataset = dataset.sample(frac=1,random_state=rand)
    
    size = dataset.shape[0]
    k = fold
    step = math.floor(size/k) # size for each fold
    
    R2_c = []
    MSE_c = []
    R2_s = []
    MSE_s = []
    
    variablesC = dataset.columns.drop('NumberOfCustomers')
    variablesC = variablesC.drop('NumberOfSales')
    
      
    for i in range(0,dataset.shape[0],step+1):
        
        left_data = dataset[0:i]
        test_data = dataset[i:i+step]                         #train_data < test_data > train_data
        right_data = dataset[i+step+1:dataset.shape[0]]
        
        
        train_data = left_data.append(right_data)
        train_data = train_data.reset_index(drop=True)

        
        #train_data, test_data = model_selection.train_test_split(dataset,train_size = test_size =0.3, random_state=rand)
    
        var = train_data[variablesC]
        tar = train_data['NumberOfSales']
    
        model1 = RandomForestRegressor(random_state=rand,n_jobs =2,min_samples_leaf=5) #model 1 predice customers
        model1.set_params(n_estimators=trees)
        model1.fit(var, tar)
    
    
        ###############################################################################################testing step
    

        NumOfSales = test_data['NumberOfSales']     
    
        first_test = test_data[variablesC]  
    
        Salesp = model1.predict(first_test) 
    
        R2_s.append(r2_score(NumOfSales,Salesp)) 
        MSE_s.append(mean_squared_error(NumOfSales,Salesp)) 
    
    
    return  R2_s , MSE_s

# 50 all

In [6]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data.drop(labels='NumberOfCustomers',axis=1)
var = var.drop(labels='NumberOfSales',axis=1)
#else:
#    var = train_data[cust_i]

model1 = RandomForestRegressor(random_state=12345,n_jobs=2) #model 1 predice customers
model1.set_params(n_estimators=50)
model1.fit(var, tar)

FitCust = model1.predict(var)

tar = train_data['NumberOfSales']
var = train_data.drop(labels='NumberOfSales',axis=1)
var['NumberOfCustomers'] = FitCust

model2 = RandomForestRegressor(random_state=12345,n_jobs =2) #model 2 predice sales
model2.set_params(n_estimators=50)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales


first_test = test_data.drop(labels='NumberOfSales',axis=1)
first_test = first_test.drop(labels='NumberOfCustomers',axis=1)



Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    
second_test = test_data.drop(labels='NumberOfSales',axis=1)
#second_test = test_data[sales_i] #un po inutile solo per chiarezza
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
R2_s

0.7930716530025319

## 50 all direct(without double regr)

In [7]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfSales']
var = train_data.drop(labels='NumberOfCustomers',axis=1)
var = var.drop(labels='NumberOfSales',axis=1)


model1 = RandomForestRegressor(random_state=12345,oob_score=True) 
model1.set_params(n_estimators=50)
model1.fit(var, tar)
model1.oob_score_

0.8159499827080461

# 50 LassoFS

In [8]:
cust_i = pd.Index(['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'November',
       'December', 'Wednesday', 'Friday', 'Saturday', 'Monday', 'Thursday',
       'Snow', 'Fog'])
sales_i = pd.Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'WithNonFoodDepartment', 'Region0',
       'Region1', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December',
       'Tuesday', 'Wednesday', 'Friday', 'Monday', 'Thursday', 'Sunday',
       'Snow', 'Fog', 'Nothing', 'Rain', 'NumberOfCustomers'])

In [9]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data[cust_i]


model1 = RandomForestRegressor(random_state=12345) #model 1 predice customers
model1.set_params(n_estimators=50)
model1.fit(var, tar)

FitCust = model1.predict(var)

tar = train_data['NumberOfSales']
var = train_data[sales_i]
var['NumberOfCustomers'] = FitCust

model2 = RandomForestRegressor(random_state=12345) #model 2 predice sales
model2.set_params(n_estimators=50)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[cust_i]
    


Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    

second_test = test_data[sales_i]
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
MSE_s = mean_squared_error(NumOfSales,Salesp)
R2_s

0.7816388857147238

## 50 LassoFS direct(without double regr)

In [10]:
direct_features = pd.Index(['HasPromotions', 'NearestCompetitor', 'Region_GDP',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'Region0', 'Region2', 'Region3',
       'Region4', 'Region5', 'Region6', 'Region8', 'Region9', 'January',
       'February', 'March', 'April', 'May', 'June', 'September', 'October',
       'November', 'December', 'Tuesday', 'Wednesday', 'Friday', 'Saturday',
       'Monday', 'Snow', 'Fog', 'Rain'])

In [11]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfSales']
var = train_data[direct_features]

model1 = RandomForestRegressor(random_state=12345) 
model1.set_params(n_estimators=50)
model1.fit(var, tar)
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[direct_features]
    


Salesp = model1.predict(first_test) #model1 predice cust

    
R2_s = r2_score(NumOfSales,Salesp)
R2_s

0.7967498359482068

## 50 RFFS direct(without double regr)

In [6]:
direct_features = pd.Index(['NearestCompetitor', 'HasPromotions', 'HyperMarket', 'General',
       'Region3', 'December', 'Monday', 'Region_GDP', 'ShoppingCenter',
       'Saturday', 'StandardMarket', 'Mean_Sea_Level_PressurehPa',
       'Region_PopulationK', 'SuperMarket', 'Region_AreaKM2',
       'Mean_VisibilityKm', 'Mean_Humidity', 'Mean_Dew_PointC',
       'Mean_TemperatureC', 'H', 'Mean_Wind_SpeedKm_h'])

In [7]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfSales']
var = train_data[direct_features]

model1 = RandomForestRegressor(random_state=12345) 
model1.set_params(n_estimators=50)
model1.fit(var, tar)
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[direct_features]
    


Salesp = model1.predict(first_test) #model1 predice cust

    
R2_s = r2_score(NumOfSales,Salesp)
R2_s

0.7877040563702649

# 50 RFFS

In [20]:
cust_i = pd.Index(['NearestCompetitor', 'ShoppingCenter', 'Region3', 'HasPromotions',
       'StandardMarket', 'Region7', 'General', 'Region_AreaKM2',
       'WithNonFoodDepartment', 'SuperMarket', 'Region_GDP', 'HyperMarket',
       'Monday', 'December', 'Saturday', 'Region8', 'Region_PopulationK',
       'Region0', 'Region9', 'Region2'])
sales_i = pd.Index(['NumberOfCustomers', 'HasPromotions', 'NearestCompetitor',
       'StandardMarket', 'General', 'Monday', 'December', 'Region3',
       'Region_AreaKM2', 'Region_PopulationK', 'Region_GDP', 'Region9',
       'HyperMarket', 'Region2', 'Saturday', 'WithNonFoodDepartment',
       'ShoppingCenter', 'SuperMarket', 'Region6', 'Region0'])

In [21]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data[cust_i]


model1 = RandomForestRegressor(random_state=12345) #model 1 predice customers
model1.set_params(n_estimators=50)
model1.fit(var, tar)

FitCust = model1.predict(var)

tar = train_data['NumberOfSales']
var = train_data[sales_i]
var['NumberOfCustomers'] = FitCust

model2 = RandomForestRegressor(random_state=12345) #model 2 predice sales
model2.set_params(n_estimators=50)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[cust_i]
    


Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    

second_test = test_data[sales_i]
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
MSE_s = mean_squared_error(NumOfSales,Salesp)
R2_s

0.8114839653427683

# All below without fitting the second model with the predict

# 50 all

In [67]:
r2c,msec,r2s,mse_s = mycvRF(train,3,50,12345,[],[],True)
r2s

[0.8139799993268064, 0.8243747111675138, 0.826301038377639]

# 50 LassoFS

In [55]:
cust_i = pd.Index(['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'November',
       'December', 'Wednesday', 'Friday', 'Saturday', 'Monday', 'Thursday',
       'Snow', 'Fog'])
sales_i = pd.Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'WithNonFoodDepartment', 'Region0',
       'Region1', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December',
       'Tuesday', 'Wednesday', 'Friday', 'Monday', 'Thursday', 'Sunday',
       'Snow', 'Fog', 'Nothing', 'Rain', 'NumberOfCustomers'])

In [70]:
r2c,msec,r2s,mse_s = mycvRF(train,3,50,12345,cust_i,sales_i,False)
r2s

[0.8030916826122096, 0.8155665952380902, 0.8174717573811381]

# 50 RFFS

In [18]:
cust_i = pd.Index(['NearestCompetitor', 'ShoppingCenter', 'Region3', 'HasPromotions',
       'StandardMarket', 'Region7', 'General', 'Region_AreaKM2',
       'WithNonFoodDepartment', 'SuperMarket', 'Region_GDP', 'HyperMarket',
       'Monday', 'December', 'Saturday', 'Region8', 'Region_PopulationK',
       'Region0', 'Region9', 'Region2'])
sales_i = pd.Index(['NumberOfCustomers', 'HasPromotions', 'NearestCompetitor',
       'StandardMarket', 'General', 'Monday', 'December', 'Region3',
       'Region_AreaKM2', 'Region_PopulationK', 'Region_GDP', 'Region9',
       'HyperMarket', 'Region2', 'Saturday', 'WithNonFoodDepartment',
       'ShoppingCenter', 'SuperMarket', 'Region6', 'Region0'])

In [19]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data[cust_i]


model1 = RandomForestRegressor(random_state=12345) #model 1 predice customers
model1.set_params(n_estimators=50)
model1.fit(var, tar)


tar = train_data['NumberOfSales']
var = train_data[sales_i]

model2 = RandomForestRegressor(random_state=12345) #model 2 predice sales
model2.set_params(n_estimators=50)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[cust_i]
    


Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    

second_test = test_data[sales_i]
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
MSE_s = mean_squared_error(NumOfSales,Salesp)
R2_s

0.8022241296313875

# 150 all

In [12]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data.drop(labels='NumberOfCustomers',axis=1)
var = var.drop(labels='NumberOfSales',axis=1)
#else:
#    var = train_data[cust_i]

model1 = RandomForestRegressor(random_state=12345) #model 1 predice customers
model1.set_params(n_estimators=150)
model1.fit(var, tar)

FitCust = model1.predict(var)

tar = train_data['NumberOfSales']
var = train_data.drop(labels='NumberOfSales',axis=1)
var['NumberOfCustomers'] = FitCust

model2 = RandomForestRegressor(random_state=12345) #model 2 predice sales
model2.set_params(n_estimators=150)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales


first_test = test_data.drop(labels='NumberOfSales',axis=1)
first_test = first_test.drop(labels='NumberOfCustomers',axis=1)



Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    
second_test = test_data.drop(labels='NumberOfSales',axis=1)
#second_test = test_data[sales_i] #un po inutile solo per chiarezza
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
R2_s

0.7949239288010542

## 150 all direct(without double regr)

In [71]:
model1 = RandomForestRegressor(random_state=12345) 
tar = train['NumberOfSales']
var = train.drop(labels='NumberOfCustomers',axis=1)
var = var.drop(labels='NumberOfSales',axis=1)
model1.set_params(n_estimators=150)
#R2s_s.append(cross_val_score(model1, var, tar, scoring="r2", cv =KFold(3, shuffle=True, random_state=12345)))
cross_val_score(model1, var, tar, scoring="r2", cv =KFold(3, shuffle=True, random_state=12345))

array([0.81351023, 0.82079829, 0.82293227])

In [5]:
r2,mse = mycvRFDirect(train,3,150,12345)

## 150 RFFS direct(without double regr)

In [23]:
direct_features = pd.Index(['NearestCompetitor', 'HasPromotions', 'HyperMarket',
       'General', 'Region3', 'December',
       'Monday'])

In [24]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfSales']
var = train_data[direct_features]

model1 = RandomForestRegressor(random_state=12345) 
model1.set_params(n_estimators=150)
model1.fit(var, tar)
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[direct_features]
    


Salesp = model1.predict(first_test) #model1 predice cust

    
R2_s = r2_score(NumOfSales,Salesp)
R2_s

0.7201857868035306

# 150 LassoFS

In [13]:
cust_i = pd.Index(['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'November',
       'December', 'Wednesday', 'Friday', 'Saturday', 'Monday', 'Thursday',
       'Snow', 'Fog'])
sales_i = pd.Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'WithNonFoodDepartment', 'Region0',
       'Region1', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December',
       'Tuesday', 'Wednesday', 'Friday', 'Monday', 'Thursday', 'Sunday',
       'Snow', 'Fog', 'Nothing', 'Rain', 'NumberOfCustomers'])

In [14]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3,random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data[cust_i]


model1 = RandomForestRegressor(random_state=12345,n_jobs = 2) #model 1 predice customers
model1.set_params(n_estimators=150)
model1.fit(var, tar)

FitCust = model1.predict(var)

tar = train_data['NumberOfSales']
var = train_data[sales_i]
var['NumberOfCustomers'] = FitCust

model2 = RandomForestRegressor(random_state=12345,n_jobs = 2) #model 2 predice sales
model2.set_params(n_estimators=150)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[cust_i]
    


Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    

second_test = test_data[sales_i]
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
MSE_s = mean_squared_error(NumOfSales,Salesp)
R2_s

0.7823853984137392

## 150 LassoFS direct(without double regr)

In [15]:
direct_features = pd.Index(['HasPromotions', 'NearestCompetitor', 'Region_GDP',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'Region0', 'Region2', 'Region3',
       'Region4', 'Region5', 'Region6', 'Region8', 'Region9', 'January',
       'February', 'March', 'April', 'May', 'June', 'September', 'October',
       'November', 'December', 'Tuesday', 'Wednesday', 'Friday', 'Saturday',
       'Monday', 'Snow', 'Fog', 'Rain'])

In [16]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfSales']
var = train_data[direct_features]

model1 = RandomForestRegressor(random_state=12345) 
model1.set_params(n_estimators=50)
model1.fit(var, tar)
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[direct_features]
    


Salesp = model1.predict(first_test) #model1 predice cust

    
R2_s = r2_score(NumOfSales,Salesp)
R2_s

0.7967498359482068

# 150 RFFS

In [22]:
cust_i = pd.Index(['NearestCompetitor', 'ShoppingCenter', 'Region3', 'HasPromotions',
       'StandardMarket', 'Region7', 'General', 'Region_AreaKM2',
       'WithNonFoodDepartment', 'SuperMarket', 'Region_GDP', 'HyperMarket',
       'Monday', 'December', 'Saturday', 'Region8', 'Region_PopulationK',
       'Region0', 'Region9', 'Region2'])
sales_i = pd.Index(['NumberOfCustomers', 'HasPromotions', 'NearestCompetitor',
       'StandardMarket', 'General', 'Monday', 'December', 'Region3',
       'Region_AreaKM2', 'Region_PopulationK', 'Region_GDP', 'Region9',
       'HyperMarket', 'Region2', 'Saturday', 'WithNonFoodDepartment',
       'ShoppingCenter', 'SuperMarket', 'Region6', 'Region0'])

In [23]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data[cust_i]


model1 = RandomForestRegressor(random_state=12345) #model 1 predice customers
model1.set_params(n_estimators=150)
model1.fit(var, tar)

FitCust = model1.predict(var)

tar = train_data['NumberOfSales']
var = train_data[sales_i]
var['NumberOfCustomers'] = FitCust

model2 = RandomForestRegressor(random_state=12345) #model 2 predice sales
model2.set_params(n_estimators=150)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[cust_i]
    


Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    

second_test = test_data[sales_i]
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
MSE_s = mean_squared_error(NumOfSales,Salesp)
R2_s

0.811559263107862

# All below without fitting the second model with the predict

# 150 all

In [ ]:
r2c,msec,r2s,mse_s = mycvRF(train,3,150,12345,[],[],True)
r2s

# 150 LassoFS

In [ ]:
cust_i = pd.Index(['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'November',
       'December', 'Wednesday', 'Friday', 'Saturday', 'Monday', 'Thursday',
       'Snow', 'Fog'])
sales_i = pd.Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'WithNonFoodDepartment', 'Region0',
       'Region1', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December',
       'Tuesday', 'Wednesday', 'Friday', 'Monday', 'Thursday', 'Sunday',
       'Snow', 'Fog', 'Nothing', 'Rain', 'NumberOfCustomers'])

In [ ]:
r2c,msec,r2s,mse_s = mycvRF(train,3,150,12345,cust_i,sales_i,False)
r2s

# 150 RFFS

In [24]:
cust_i = pd.Index(['NearestCompetitor', 'ShoppingCenter', 'Region3', 'HasPromotions',
       'StandardMarket', 'Region7', 'General', 'Region_AreaKM2',
       'WithNonFoodDepartment', 'SuperMarket', 'Region_GDP', 'HyperMarket',
       'Monday', 'December', 'Saturday', 'Region8', 'Region_PopulationK',
       'Region0', 'Region9', 'Region2'])
sales_i = pd.Index(['NumberOfCustomers', 'HasPromotions', 'NearestCompetitor',
       'StandardMarket', 'General', 'Monday', 'December', 'Region3',
       'Region_AreaKM2', 'Region_PopulationK', 'Region_GDP', 'Region9',
       'HyperMarket', 'Region2', 'Saturday', 'WithNonFoodDepartment',
       'ShoppingCenter', 'SuperMarket', 'Region6', 'Region0'])

In [25]:
train_data, test_data = model_selection.train_test_split(train,test_size =0.3, random_state=12345)

tar = train_data['NumberOfCustomers']
var = train_data[cust_i]


model1 = RandomForestRegressor(random_state=12345) #model 1 predice customers
model1.set_params(n_estimators=150)
model1.fit(var, tar)


tar = train_data['NumberOfSales']
var = train_data[sales_i]

model2 = RandomForestRegressor(random_state=12345) #model 2 predice sales
model2.set_params(n_estimators=150)
model2.fit(var, tar)
    
    ###############################################################################################testing step
    
NumOfCust = test_data['NumberOfCustomers']   #metto da parte i cust
NumOfSales = test_data['NumberOfSales']      #metto da parte le sales



first_test = test_data[cust_i]
    


Custp = model1.predict(first_test) #model1 predice cust
    
R2_c = r2_score(NumOfCust,Custp) #calcolo R2 dei cust
MSE_c = mean_squared_error(NumOfCust,Custp) #calcolo MSE dei cust
    

second_test = test_data[sales_i]
second_test['NumberOfCustomers'] = Custp #metto i cust predetti dal model1 nel test per predire le vendite future
    
Salesp = model2.predict(second_test) #model2 predice sales
    
R2_s = r2_score(NumOfSales,Salesp)
MSE_s = mean_squared_error(NumOfSales,Salesp)
R2_s

0.8024608714349151

# Comparing best results

In [2]:
R2s_s = [[0.8210592430007686, 0.830533519620758, 0.8339257920035679],
        [0.8156093645619071, 0.8263774735530535, 0.8297193934502831],
        [0.8266803778224731, 0.8345156712098017, 0.837037448176467],
        [0.8219272767777451, 0.8314003014987681, 0.8347551571574651],
        [0.8163929841495856, 0.8271603810578703, 0.8307763602522618]]

In [3]:
all_50 = R2s_s[0]
lassoFs_50 = R2s_s[1]
allDirect_150 = R2s_s[2]
all_150 = R2s_s[3]
lassoFs_150 = R2s_s[4]

In [4]:
print("Sales:all features and 50 trees      \t%.1f\t%.6f" % (100.0*np.average(all_50), 100.0*np.std(all_50)))

Sales:all features and 50 trees      	82.9	0.544485


In [5]:
print("Sales:Lasso features and 50 trees      \t%.1f\t%.6f" % (100.0*np.average(lassoFs_50), 100.0*np.std(lassoFs_50)))

Sales:Lasso features and 50 trees      	82.4	0.602046


In [6]:
print("Sales:all features witout double regression and 150 trees      \t%.1f\t%.6f" % (100.0*np.average(allDirect_150), 100.0*np.std(allDirect_150)))

Sales:all features witout double regression and 150 trees      	83.3	0.440984


In [7]:
print("Sales:all features and 150 trees      \t%.1f\t%.6f" % (100.0*np.average(all_150), 100.0*np.std(all_150)))

Sales:all features and 150 trees      	82.9	0.543188


In [8]:
print("Sales:Lasso features and 150 trees      \t%.1f\t%.6f" % (100.0*np.average(lassoFs_150), 100.0*np.std(lassoFs_150)))

Sales:Lasso features and 150 trees      	82.5	0.610913
